# Inteligencia Artificial
# Clase 14 - Deep Learning para Computer Vision 2

## Transfer Learning

En la práctica anterior, vimos cómo la *data augmentation* nos permitió combatir el sobreajuste de una red neuronal convolucional entrenada sobre un dataset relativamente pequeño, de apenas 2000 muestras (1000 por cada clase). Gracias a esta técnica de regularización, pudimos mejorar en 10 puntos porcentuales el accuracy de nuestro clasificador de imágenes de perros y gatos.

En esta notebook, presentaremos otro enfoque común y altamente efectivo en *deep learning* que consiste en usar una red preentrenada. Esta técnica, conocida como *transfer learning*, nos permite aprovechar un modelo que ya ha sido entrenado previamente en un gran conjunto de datos (por ejemplo, el dataset de [ImageNet](http://www.image-net.org/)) y reutilizarlo en el contexto de un problema específico. Si el conjunto de datos con el que se entrenó el modelo es lo suficientemente grande y general, entonces la jerarquía espacial de *features* aprendida por la red puede actuar efectivamente como un modelo genérico del mundo visual y, por lo tanto, las *features* pueden resultar útiles para muchos problemas de *computer vision* diferentes, a pesar de que éstos sean completamente diferentes a la tarea original.

Veremos que Keras ofrece en el módulo de aplicaciones distintos modelos destacados, previamente entrenadas sobre ImageNet, para que podamos reutilizarlos y aplicarlos a nuestros propios datasets. En este caso, trabajaremos con el modelo [VGG16](https://arxiv.org/abs/1409.1556) para mejorar la *performance* de nuestro clasificador de perros y gatos.

<img src="https://distilledai.com/wp-content/uploads/2020/04/cat-vs-dog.jpeg" width=500 />

El contenido de esta notebook se basa mayormente en un ejemplo del [capítulo 5 del libro Deep Learning with Python](https://livebook.manning.com/book/deep-learning-with-python/chapter-5/), de François Chollet (2017).

In [1]:
!mkdir -p ~/.kaggle
! echo '{"username":"mggaska","key":"10bb9c3aaa775f3385c5c7e3a17a3eaf"}' >> kaggle.json

In [2]:
!mv kaggle.json ~/.kaggle/

In [3]:
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
cat ~/.kaggle/kaggle.json

{"username":"mggaska","key":"10bb9c3aaa775f3385c5c7e3a17a3eaf"}


In [5]:
!mkdir kaggle_original_data

mkdir: cannot create directory ‘kaggle_original_data’: File exists


In [6]:
!pip install kaggle

You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
%%bash
cd kaggle_original_data
kaggle competitions download -c dogs-vs-cats
unzip dogs-vs-cats.zip
unzip test1.zip
unzip train.zip

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dogs-vs-cats.zip


replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip tes]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [t1.zip]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip tra]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [in.zip]
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [8]:
# Estructuramos los directorios de trabajo
import os, shutil

# El path al directorio donde se descomprimió el dataset
original_dataset_dir = './kaggle_original_data/train'

# El directorio donde guardaremos el más pequeño
base_dir = './cats_and_dogs_small'
os.makedirs(base_dir, exist_ok=True)

# Directorios para los splits de
# entrenamiento, validación y test
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

# Directorio con imágenes de entrenamiento de gatos
train_cats_dir = os.path.join(train_dir, 'cats')
os.makedirs(train_cats_dir, exist_ok=True)

# Directorio con imágenes de entrenamiento de perros
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_dogs_dir, exist_ok=True)

# Directorio con las imágenes de validación de gatos
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.makedirs(validation_cats_dir, exist_ok=True)

# Directorio con las imágenes de validación de perros
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.makedirs(validation_dogs_dir, exist_ok=True)

# Directorio con imágenes de test de gatos
test_cats_dir = os.path.join(test_dir, 'cats')
os.makedirs(test_cats_dir, exist_ok=True)

# Directorio con las imágenes de test de perros
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.makedirs(test_dogs_dir, exist_ok=True)

# Copiamos las primeras 1000 imágenes de gatos a train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Las siguientes 500 a validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copiamos las siguientes 500 a test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
 # Copiamos las primeras 1000 imágenes de perros a train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Las siguientes 500 a validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Y las siguientes 500 a test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

## VGG16

Vamos a utilizar la arquitectura VGG16, desarrollada por Karen Simonyan y Andrew
Zisserman en el 2014. Este modelo es una *convnet* relativamente simple en su estructura, no muy diferente a las redes convolucionales con las que trabajamos hasta ahora, aunque sí mucho más profunda.

<img src="https://www.researchgate.net/profile/Jose_Cano31/publication/327070011/figure/fig1/AS:660549306159105@1534498635256/VGG-16-neural-network-architecture.png"/>

Podemos importar la clase `VGG16` del módulo [`applications`](https://keras.io/applications/) de Keras y generar una instancia del modelo:

In [9]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Especificamos tres argumentos en el constructor:
- `weights` define los pesos con los que inicializar el modelo. Puede ser `imagenet`, en caso de que deseemos utilizar el modelo preentrenado en este dataset, o `None`, si deseamos trabajar únicamente con la arquitectura de la red pero inicializar los pesos de manera aleatoria.
- `include_top` se refiere a la inclusión (o no) de las capas densamente conectadas que se encuentran al final de la red original y que permiten clasificar las 1000 clases de ImageNet. Como debemos resolver una clasificación binaria, no necesitaremos incluir las capas densas en nuestro caso. Por lo tanto, sólo trabajaremos con la base convolucional del modelo VGG16. 
- `input_shape` corresponde a la forma del tensor de imágenes con el que alimentaremos la red neuronal.

Veamos la base convolucional de VGG16:

In [10]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

Hay dos maneras en que podemos usar una red preentrenada: *feature extraction* y *fine-tuning*. Vamos a cubrir ambas, comenzando por *feature extraction*.

## Feature extraction

***Feature extraction* consiste en usar las representaciones aprendidas por una red preentrenada para extraer atributos interesantes de nuevas muestras. Estas *features* son luego utilizadas por un nuevo clasificador, que entrenamos desde cero.**

Notemos que el mapa de features final de la VGG16 tiene forma `(4, 4, 512)`. Ésta será la entrada de un clasificador densamente conectado.

En este punto tenemos dos maneras para proceder:

* Correr la base convolucional sobre nuestro dataset, guardar su salida en una matriz Numpy en el disco, y luego usar esta información como entrada para un clasificador densamente conectado e independiente similar a los que ya vimos. 
Esta solución es rápida y barata de ejecutar, porque sólo requiere ejecutar la base convolucional una vez para cada imagen de entrada, y la base convolucional es, por mucho, la parte más costosa del *pipeline*. Pero por la misma razón, esta técnica no permite hacer *data augmentation*.

* Extender el modelo que tenemos agregando capas densas en la parte superior, y entrenar la red de punta a punta sobre los datos de entrada. Esto nos permitirá usar *data augmentation*, porque cada imagen de entrada pasa por la base convolucional cada vez que es vista por el modelo. Pero por la misma razón, esta técnica es mucho más costosa que la primero.

### Feature extraction rápida sin data augmentation

Primero veremos la técnica de *feature extraction* rápida:

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = './cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
  
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Notar que este generador devuelve data indefinidamente en un loop,
            # debemos cortarlo con un "break" despues de que cada imagen haya sido vista una vez
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Las *features* extraídas son de forma `(samples, 4, 4, 512)`. Con ellas vamos a alimentar un clasificador densamente conectado, así que primero tenemos que aplanarlas a la forma `(samples, 8192)`:

In [12]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Ahora definimos el clasificador y entrenamos el modelo:

In [13]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])



In [14]:
%%timeit
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 663us/sample - loss: 0.6117 - acc: 0.6635 - val_loss: 0.4591 - val_acc: 0.8340
Epoch 2/30
2000/2000 [==============================] - 1s 376us/sample - loss: 0.4335 - acc: 0.8055 - val_loss: 0.3719 - val_acc: 0.8650
Epoch 3/30
2000/2000 [==============================] - 1s 374us/sample - loss: 0.3583 - acc: 0.8560 - val_loss: 0.3453 - val_acc: 0.8530
Epoch 4/30
2000/2000 [==============================] - 1s 377us/sample - loss: 0.3230 - acc: 0.8715 - val_loss: 0.3064 - val_acc: 0.8810
Epoch 5/30
2000/2000 [==============================] - 1s 373us/sample - loss: 0.2900 - acc: 0.8810 - val_loss: 0.2867 - val_acc: 0.8920
Epoch 6/30
2000/2000 [==============================] - 1s 379us/sample - loss: 0.2690 - acc: 0.8990 - val_loss: 0.2766 - val_acc: 0.8890
Epoch 7/30
2000/2000 [==============================] - 1s 378us/sample - loss: 0.2468 - acc: 0.8975 - val_loss: 0.2651 - va

2000/2000 [==============================] - 1s 387us/sample - loss: 0.0276 - acc: 0.9960 - val_loss: 0.2775 - val_acc: 0.8980
Epoch 30/30
2000/2000 [==============================] - 1s 391us/sample - loss: 0.0267 - acc: 0.9980 - val_loss: 0.2800 - val_acc: 0.8970
Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 389us/sample - loss: 0.0281 - acc: 0.9960 - val_loss: 0.2789 - val_acc: 0.8990
Epoch 2/30
2000/2000 [==============================] - 1s 382us/sample - loss: 0.0253 - acc: 0.9980 - val_loss: 0.2843 - val_acc: 0.8980
Epoch 3/30
2000/2000 [==============================] - 1s 383us/sample - loss: 0.0217 - acc: 0.9990 - val_loss: 0.2835 - val_acc: 0.8990
Epoch 4/30
2000/2000 [==============================] - 1s 385us/sample - loss: 0.0239 - acc: 0.9970 - val_loss: 0.2881 - val_acc: 0.8970
Epoch 5/30
2000/2000 [==============================] - 1s 388us/sample - loss: 0.0220 - acc: 0.9975 - val_loss: 0.2887 - val_acc: 0.8

2000/2000 [==============================] - 1s 392us/sample - loss: 0.0017 - acc: 1.0000 - val_loss: 0.4177 - val_acc: 0.9030
Epoch 28/30
2000/2000 [==============================] - 1s 384us/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.4297 - val_acc: 0.8970
Epoch 29/30
2000/2000 [==============================] - 1s 390us/sample - loss: 0.0016 - acc: 1.0000 - val_loss: 0.4394 - val_acc: 0.8980
Epoch 30/30
2000/2000 [==============================] - 1s 388us/sample - loss: 0.0016 - acc: 1.0000 - val_loss: 0.4578 - val_acc: 0.8890
Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 1s 390us/sample - loss: 0.0016 - acc: 1.0000 - val_loss: 0.4370 - val_acc: 0.8990
Epoch 2/30
2000/2000 [==============================] - 1s 389us/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 0.4496 - val_acc: 0.8960
Epoch 3/30
2000/2000 [==============================] - 1s 392us/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 0.4423 - val_acc: 0

2000/2000 [==============================] - 1s 393us/sample - loss: 1.0504e-04 - acc: 1.0000 - val_loss: 0.6069 - val_acc: 0.8940
Epoch 25/30
2000/2000 [==============================] - 1s 398us/sample - loss: 1.1446e-04 - acc: 1.0000 - val_loss: 0.6023 - val_acc: 0.8990
Epoch 26/30
2000/2000 [==============================] - 1s 407us/sample - loss: 7.9907e-05 - acc: 1.0000 - val_loss: 0.6254 - val_acc: 0.8950
Epoch 27/30
2000/2000 [==============================] - 1s 399us/sample - loss: 9.8319e-05 - acc: 1.0000 - val_loss: 0.6348 - val_acc: 0.8940
Epoch 28/30
2000/2000 [==============================] - 1s 414us/sample - loss: 6.8245e-05 - acc: 1.0000 - val_loss: 0.6373 - val_acc: 0.8930
Epoch 29/30
2000/2000 [==============================] - 1s 417us/sample - loss: 8.8194e-05 - acc: 1.0000 - val_loss: 0.6405 - val_acc: 0.8940
Epoch 30/30
2000/2000 [==============================] - 1s 410us/sample - loss: 7.2060e-05 - acc: 1.0000 - val_loss: 0.6469 - val_acc: 0.8940
Train on 20

Epoch 21/30
2000/2000 [==============================] - 1s 380us/sample - loss: 6.7349e-06 - acc: 1.0000 - val_loss: 0.7820 - val_acc: 0.8940
Epoch 22/30
2000/2000 [==============================] - 1s 380us/sample - loss: 1.1302e-05 - acc: 1.0000 - val_loss: 0.7806 - val_acc: 0.8960
Epoch 23/30
2000/2000 [==============================] - 1s 378us/sample - loss: 7.7359e-06 - acc: 1.0000 - val_loss: 0.7723 - val_acc: 0.8970
Epoch 24/30
2000/2000 [==============================] - 1s 383us/sample - loss: 7.2002e-06 - acc: 1.0000 - val_loss: 0.8148 - val_acc: 0.8930
Epoch 25/30
2000/2000 [==============================] - 1s 384us/sample - loss: 5.9774e-06 - acc: 1.0000 - val_loss: 0.8038 - val_acc: 0.8930
Epoch 26/30
2000/2000 [==============================] - 1s 377us/sample - loss: 6.6924e-06 - acc: 1.0000 - val_loss: 0.8123 - val_acc: 0.8920
Epoch 27/30
2000/2000 [==============================] - 1s 378us/sample - loss: 6.0310e-06 - acc: 1.0000 - val_loss: 0.8206 - val_acc: 0.8910

## Guardamos el modelo

Para probar implementar este modelo en el Tensorflow Serving Container guardamos el modelo y los test features para la invocación

In [15]:
model.save('./model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./model/assets


In [16]:
import numpy as np
np.save('test_features.npy', test_features) 
np.save('test_labels.npy', test_labels) 